* Use paraphrase-albert-small-v2 instead

# Sentence Embeddings using Siamese BERT-Networks/Sentence Transformers
* Also TSDAE. pretraining? 
---

The Sentence Transformer library is available on [pypi](https://pypi.org/project/sentence-transformers/) and [github](https://github.com/UKPLab/sentence-transformers). The library implements code from the ACL 2019 paper entitled "[Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks](https://www.aclweb.org/anthology/D19-1410.pdf)" by Nils Reimers and Iryna Gurevych.

* https://github.com/UKPLab/sentence-transformers/blob/master/examples/unsupervised_learning/TSDAE/train_askubuntu_tsdae.py
* https://www.sbert.net/examples/unsupervised_learning/TSDAE/README.html#tsdae-as-pre-training-task
* https://www.sbert.net/docs/quickstart.html

This version also combined white/black pairs and trains over a single column/text
* https://datascience.stackexchange.com/questions/39345/how-to-replace-a-part-string-value-of-a-column-using-another-column
* NOTE: Some cards lack a "____" - need to handle them 


In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

## Install Sentence Transformer Library

In [2]:
# # Install the library using pip
# !pip3 install sentence-transformers scikit-learn tensorflow -U
# # !pip3 install sentence-transformers scikit-learn tensorflow tensorflow-text tf-models-official -U
# # !pip3 install scikit-learn  -U
# # !pip3 install nltk scikit-learn  -U

# # import nltk
# # nltk.download('punkt')

In [4]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, roc_auc_score  

# https://stackoverflow.com/questions/53784971/how-to-disable-convergencewarning-using-sklearn
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
from sklearn.model_selection import train_test_split


In [5]:
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses, SentencesDataset ## https://www.sbert.net/docs/package_reference/losses.html + SentencesDataset
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, SentencesDataset, losses ## MultipleNegativesRankingLoss
## https://www.sbert.net/docs/package_reference/losses.html#multiplenegativesrankingloss
# from sentence_transformers.readers import InputExample
import logging
import os
import gzip
from torch.utils.data import DataLoader
from datetime import datetime
import sys

In [6]:
model_name = "paraphrase-albert-small-v2"#"all-MiniLM-L12-v2" #"all-MiniLM-L6-v2" # "all-MiniLM-L12-v2"
# model_name = "/content/drive/MyDrive/research/cah/cah_tsdae-model" # prev
min_cooccurences = 1 #4 # filter sentences for pairs that occurred at least k times. min 5: 200K. min 1: 1.9M

ONE_COL_DATA_FORMAT = True#False
USE_TEXT_COLS =  ["text"]#["text",	"white_card_text"]#["black_card_text",	"white_card_text"]

FAST_RUN = False#True#False

## Load the sBERT Model

* Default , later try pretrained+ 

In [7]:
# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

# model = SentenceTransformer('bert-base-nli-mean-tokens')
# model = SentenceTransformer("nli-distilroberta-base-v2")
## # https://www.sbert.net/docs/pretrained_models.html
# model = SentenceTransformer("paraphrase-MiniLM-L6-v2") ## "paraphrase-MiniLM-L12-v2"
# model = SentenceTransformer("paraphrase-MiniLM-L12-v2")
model = SentenceTransformer(model_name)
# model = SentenceTransformer("all-MiniLM-L12-v2")

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/827 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/245 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/465 [00:00<?, ?B/s]

In [8]:
## copied from another notebook; may not be most up to date??

def eval_preds(df_test,preds):
    df2 = df_test[["fake_round_id","won"]].copy()
    df2["preds"] = preds
    df2 = df2.sample(frac=1)
    df2["m_score"] = df2.groupby("fake_round_id")["preds"].transform("max")
    df2["correct"] = ((df2["preds"]==df2["m_score"]) &(df2["won"]>0)).astype(int)#df2.loc[df2["preds"]==df2["m_score"]
    # df2.tail(31) ## there are cases wtih multiple values with same score/rank?  - ignore for now
    df2.sort_values("preds",ascending=False,inplace=True)
    print(classification_report(df2["won"],df2["correct"]))
    print("rocAUC:",round(roc_auc_score(df2["won"],df2["preds"]),4))
    print(f'top 1 acc by round: {100*df2.groupby("fake_round_id").head(1)["won"].mean():.2f}') # older ?
    print(f'top 2 acc by round: {100*df2.groupby("fake_round_id").head(2)["won"].mean():.2f}')# older ?
    print(f'top 3 acc by round: {100*df2.groupby("fake_round_id").head(3)["won"].mean():.2f}')# older ?
    print("top 1 acc by round:",round(df2.groupby("fake_round_id").head(1)["won"].mean(),4))
    print("top 2 acc by round:",round(df2.groupby("fake_round_id").head(2).groupby("fake_round_id")["won"].max().mean(),4))
    print("top 3 acc by round:",round(df2.groupby("fake_round_id").head(3).groupby("fake_round_id")["won"].max().mean(),4))

## Setup a Corpus

* `/content/drive/MyDrive/Research/CAH/cah_train_min7_v1.csv` - 37K
    * `/content/drive/MyDrive/Research/CAH/cah_min7_v2.csv.gz`
* `/content/drive/MyDrive/Research/CAH/cah_train_min6_v1.csv`

In [9]:
# KEEP_COLS = ["black_card_text","white_card_text","text","picks"]

In [10]:
## "/content/drive/MyDrive/Research/CAH/cah_min6_v2.csv.gz"
# "/content/drive/MyDrive/Research/CAH/cah_min2_v2.csv.gz"
# "/content/drive/MyDrive/Research/CAH/cah_train_min4_v1.csv"
# df = pd.read_csv("/content/drive/MyDrive/Research/CAH/cah_min6_v2.csv.gz",usecols=["black_card_text","white_card_text","picks"])#.sample(99)
# df = pd.read_csv("/content/drive/MyDrive/Research/CAH/cah_train_min4_v1.csv",usecols=["black_card_text","white_card_text","picks"])#.sample(99)

# df = pd.read_parquet("/content/drive/MyDrive/research/cah/cah_train_games.parquet").drop(columns=["prior_white"],errors="ignore") 
df = pd.read_parquet("cah_train_games.parquet").drop(columns=["prior_white"],errors="ignore") 

if FAST_RUN:
    df = df.sort_values("fake_round_id").head(11230)
df["won"] = df["won"].astype(int)
df = df.sort_values(by="won",ascending=False) ## get picked pairs first

# df_all = df.copy() ## copy for quick eval
print(df.shape[0],"rows")

df["pair_count"] = df.groupby("text")["won"].transform("count") ## can be used to filter sentences occurring less than k times
print(df[["won","pair_count"]].describe().round(2) )

# df = df.drop_duplicates(subset=["black_card_text","white_card_text"],keep="first")#.sample(frac=1)
print(df.shape[0])

1957080 rows
             won  pair_count
count  1957080.0  1957080.00
mean         0.1        3.53
std          0.3        1.60
min          0.0        1.00
25%          0.0        2.00
50%          0.0        3.00
75%          0.0        4.00
max          1.0       13.00
1957080


In [11]:
# %%time
# df_embeds = pd.read_parquet("cah_embed_L12.parquet") ## existing file I have locally. is it good? 
# df_embeds

In [12]:
# df_embeds.isna().max(axis=0).sum()

Test set
* Keeps round level grouping

In [13]:
# df_test = pd.read_parquet("/content/drive/MyDrive/research/cah/cah_test_games.parquet").drop(columns=["prior_white"],errors="ignore").sample(frac=1)
df_test = pd.read_parquet("cah_test_games.parquet").drop(columns=["prior_white"],errors="ignore")
df_test["won"] = df_test["won"].astype(int)

if FAST_RUN:
    df_test = df_test.sort_values("fake_round_id").head(230)
print(df_test.nunique())
print(df_test.shape[0],"rows")
df_test

fake_round_id                48928
black_card_text                580
white_card_text               2118
ID_index                        10
id_white                      2118
round_completion_seconds       705
won                              2
text                        377397
sum_won                          9
dtype: int64
489280 rows


,fake_round_id,black_card_text,white_card_text,ID_index,id_white,round_completion_seconds,won,text,sum_won
2387657,238766,What's about to take this dance floor to the n...,The NRA,8,1755,46,0,What's about to take this dance floor to the n...,1
2387651,238766,What's about to take this dance floor to the n...,Fisting,2,790,46,0,What's about to take this dance floor to the n...,0
2387653,238766,What's about to take this dance floor to the n...,Crab,4,590,46,1,What's about to take this dance floor to the n...,1
2387654,238766,What's about to take this dance floor to the n...,Slowly releasing a huge fart over the course o...,5,1591,46,0,What's about to take this dance floor to the n...,1
2387655,238766,What's about to take this dance floor to the n...,Elon Musk,6,721,46,0,What's about to take this dance floor to the n...,0
...,...,...,...,...,...,...,...,...,...
2989543,298955,Oh my god! __ killed Kenny!,Holding up the line at Walgreens by trying to ...,4,997,7613,0,Oh my god! Holding up the line at Walgreens by...,0
2989545,298955,Oh my god! __ killed Kenny!,Breastfeeding a ten-year-old,6,495,7613,0,Oh my god! Breastfeeding a ten-year-old killed...,0
2989542,298955,Oh my god! __ killed Kenny!,A bird that shits human turds,3,56,7613,0,Oh my god! A bird that shits human turds kille...,0
2989547,298955,Oh my god! __ killed Kenny!,Jerking off to a 10-second RealMedia clip,8,1077,7613,0,Oh my god! Jerking off to a 10-second RealMedi...,0


### Mean baseline priors
* By min cooccurrences of sentence pairs in in raw data
```
  1 min occ Prior Acc: 0.2044
  2 min occ Prior Acc: 0.2032
  3 min occ Prior Acc: 0.2027
  4 min occ Prior Acc: 0.2011
  5 min occ Prior Acc: 0.1922
  6 min occ Prior Acc: 0.1762
  7 min occ Prior Acc: 0.1503
```

In [14]:
try:
  for i in range(1,8):
    print(f"{i} min occ, {df_temp.shape[0]} rows")
    df_temp = df.loc[df["pair_count"] >=i].copy()
    df_white_prior = df_temp.groupby(["white_card_text"], as_index=False)["won"].mean().rename(columns={"won":"white_prior"}).set_index("white_card_text")
    df_test = df_test.join(df_white_prior,on="white_card_text",how="left")
    prior = df_test["white_prior"].mean()
    df_test["white_prior"] = df_test["white_prior"].fillna(prior)
    print("White Prior Acc: %.3f" %df_test.sort_values("white_prior",ascending=False).groupby("fake_round_id").head(1)["won"].mean()) ## 17.6% (with min 6) , 17.6% with min 3
    
  ## prior for a black-white combination - mean (freq% won), or sum (times won?) , or threshholded max? (over 2 times?)
    df_pair_prior = df_temp.groupby(["white_card_text","black_card_text"], as_index=False)["won"].sum().rename(columns={"won":"pair_prior"}).set_index(["white_card_text","black_card_text"])
    
    df_test = df_test.join(df_pair_prior,on=["white_card_text","black_card_text"],how="left")
    prior = df_test["pair_prior"].mean()
    df_test["pair_prior"] = df_test["pair_prior"].fillna(prior)
    print("Pair Prior (Only)  Acc: %.3f" %df_test.sort_values("pair_prior",ascending=False).groupby("fake_round_id").head(1)["won"].mean()) ## 17.6% (with min 6) , 17.6% with min 3
    print("White then Pair Prior Acc: %.3f" %df_test.sort_values(["white_prior","pair_prior"],ascending=False).groupby("fake_round_id").head(1)["won"].mean()) ## 17.6% (with min 6) , 17.6% with min 3
    print("Pair Prior then White Acc: %.3f" %df_test.sort_values(["pair_prior","white_prior",],ascending=False).groupby("fake_round_id").head(1)["won"].mean()) ## 17.6% (with min 6) , 17.6% with min 3
    
    df_test.drop(columns=["white_prior","pair_prior"],errors="ignore",inplace=True)
except:()

Drop duplicate instances with same out put (i.e ignore round level/ranking) 
* keep positives preferrably
* Could do : weight or filter bby # occurrences

* 22% mean win rate after this (instea of 10%)

In [15]:
if min_cooccurences>1:
  ## filter for sentence pairs occurring at least X times, regardless of label
  df = df.loc[df["pair_count"] >=min_cooccurences]
  print(df.nunique())
  print(df.shape[0],f"rows after {min_cooccurences} filter of pairs")
  df = df.drop(columns=["fake_round_id","prior_white","pair_count"],errors="ignore") # drop round id if not doing group level ranking
  try:
    df = df.sort_values(by="won",ascending=False).drop_duplicates(subset=["black_card_text","white_card_text"],keep="first")
    df_test = df_test.sort_values(by="won",ascending=False).drop_duplicates(subset=["black_card_text","white_card_text"],keep="first")
  except: print("sort by and keep first failed")

  print("mean won:",df["won"].mean())
  print(df.nunique())
  print(df.shape[0],"rows")
  df

In [16]:
df = df.sample(frac=1)
df_test = df_test.sample(frac=1)

df.reset_index(inplace=True,drop=True)
df_test.reset_index(inplace=True,drop=True)

In [17]:
# import csv
# df[[#'black_card_text',
#     # 'white_card_text',
#   'text','won']].rename(columns={'won':"target"}).to_csv(f"cah_min{min_cooccurences}.csv.gz",compression="gzip",index=False,quoting=csv.QUOTE_ALL)

#### train - eval split (if doing supervised pretraining... )

##### Data formats: 
1. 2 text cols
  * Can be white, black
  * could be merged (`text`) and white/black,
2. single joint text col
  * Merged (`text`) col



  https://www.pinecone.io/learn/train-sentence-transformers-softmax/

In [18]:
y = df["won"].values

if ONE_COL_DATA_FORMAT:
  ### 1 text col version
  ## orig:
  # X_train, X_test, y_train, y_test = train_test_split(
  #   list(df["text"].values), list(df["won"].astype(int).values), test_size=0.2, random_state=42)
## NEW/ALT - use defined train, test:
  X_train=list(df["text"].values)
  y_train = list(df["won"].values)
  X_test=list(df_test["text"].values)
  y_test = list(df_test["won"].values)

In [19]:
# # X = np.concatenate([s1_emb,s2_emb],axis=1) # BOTH 
# # X = list(df["text"].values)

# if ONE_COL_DATA_FORMAT:
#   ### 1 text col version
#   ## orig:
#   # X_train, X_test, y_train, y_test = train_test_split(
#   #   list(df["text"].values), list(df["won"].astype(int).values), test_size=0.2, random_state=42)
# ## NEW/ALT - use defined train, test:
#   X_train=list(df["text"].values)
#   y_train = list(df["won"].values)
#   X_test=list(df_test["text"].values)
#   y_test = list(df_test["won"].values)
  
#   train_samples=  [InputExample(texts=[X_train[i]],label=float(y_train[i])) for i in range(len(X_train))]
#   test_samples=  [InputExample(texts=[X_test[i]],label=float(y_test[i])) for i in range(len(X_test))]

#   ## added: copied from 2 text - expects 2 cols??
#   ## LabelAccuracyEvaluator
#   dev_eval = evaluation.BinaryClassificationEvaluator(sentences1=[x for x in X_test],sentences2 = [x for x in X_test],labels = y_test,
#                                                                           batch_size=128,show_progress_bar=True,write_csv=True)
# else:
#   ### 2 text col version ; can try different cols
#   """
#   ##ORIG:
#   X_train, X_test, y_train, y_test = train_test_split(
#     list(df[USE_TEXT_COLS].values), list(df["won"].values), test_size=0.2, random_state=42)
#   """
#   ## NEW/ALT - use defined train, test:
#   X_train=list(df[USE_TEXT_COLS].values)
#   y_train = list(df["won"].values)
#   X_test=list(df_test[USE_TEXT_COLS].values)
#   y_test = list(df_test["won"].values)
#   train_samples=  [InputExample(texts=[X_train[i][0],X_train[i][1]],label=float(y_train[i])) for i in range(len(X_train))]
#   test_samples=  [InputExample(texts=[X_test[i][0],X_test[i][1]],label=float(y_test[i])) for i in range(len(X_test))]
#   ### binary evaluator: expect list 1, list 2:
#   ## https://www.sbert.net/docs/package_reference/evaluation.html
#   ## leaky: 
#   dev_eval = evaluation.BinaryClassificationEvaluator(sentences1=[x[0] for x in X_test],sentences2 = [x[1] for x in X_test],labels = y_test,
#                                                                           batch_size=128,show_progress_bar=True,write_csv=True)
# # train_examples =  [InputExample(texts=[X_train[i]],label=float(y_train[i])) for i in range(len(X_train))]
# train_examples = X_train
# test_examples = X_test # added

* See how well unsupervised model does? 
* Check if funny combs are close or far

In [20]:
# model2 = SentenceTransformer(model_name)

In [21]:
# %%time
# ### could speed this up n^2 by getting embeddings only for unique combniations, then rejoining/merging...  (Won't work if using "Text" col)
# s1_emb = model.encode(list(df[USE_TEXT_COLS[0]].values),
#                       normalize_embeddings=True,
#                       show_progress_bar=True,
#                       batch_size=128
#                     #   convert_to_tensor=True
#                     )

# ## black cards = quesiton/prompt
# s2_emb = model.encode(list(df[USE_TEXT_COLS[1]].values),
#                       normalize_embeddings=True,
#                       show_progress_bar=True,
#                       batch_size=256
#                     #   convert_to_tensor=True
#                     )#.astype(np.float32) ## 1.5 min for 4K , with L12 miniLM, on cpu

# dot_sim = []
# for i in range (len(s1_emb)):
#     dot_sim.append(float(util.dot_score(s1_emb[i], s2_emb[i])[0]))
# print(len(dot_sim))

# df["dot_sim_score"] = dot_sim
# # df_all["dot_sim_score"] = dot_sim
# print(df[["dot_sim_score","won"]].corr())
# print("df:\n",df.groupby("won")["dot_sim_score"].mean())

# # print("df_All:")
# # print(df_all[["dot_sim_score","won"]].corr())

In [22]:
# dot_sim = []
# for i in range (len(s1_emb)):
#     dot_sim.append(float(util.dot_score(s1_emb[i], s2_emb[i])[0]))
# print(len(dot_sim))

# df["dot_sim_score"] = dot_sim
# # df_all["dot_sim_score"] = dot_sim
# print(df[["dot_sim_score","won"]].corr())
# print("df:\n",df.groupby("won")["dot_sim_score"].mean())
# print("df_all:\n",df_all.groupby("won")["dot_sim_score"].mean())

### try supervised model

* Contrastive loss?
* https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/quora_duplicate_questions/training_OnlineContrastiveLoss.py
* https://www.sbert.net/examples/training/quora_duplicate_questions/README.html#training


Can combine multiple losses:
* https://www.sbert.net/examples/training/quora_duplicate_questions/README.html

In [23]:
#As distance metric, we use cosine distance (cosine_distance = 1-cosine_similarity)
distance_metric = losses.SiameseDistanceMetric.COSINE_DISTANCE

#Negative pairs should have a distance of at least 0.3 (was 0.5 orig)
margin = 0.5

# ####  Configure the training #### 
# warmup_steps = math.ceil(len(train_dataset) * num_epochs / batch_size * 0.1) # 10% of train data for warm-up
# print("Warmup-steps: {}".format(warmup_steps))


In [24]:
# ## results before finetuning
# try:dev_eval(model) ## % with 7e-4 and min 4
# except:()

In [25]:
"""
train_dataset = SentencesDataset(train_samples, model=model)
# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size= 128,#128,
                              shuffle=True)
## OnlineContrastiveLoss. - expects pairs
# train_loss = losses.OnlineContrastiveLoss(model=model, distance_metric=distance_metric, margin=margin)
train_loss = losses.CosineSimilarityLoss(model=model) #ORIG

##ALT:
train_loss = losses.SoftmaxLoss(model=model,num_labels=1, sentence_embedding_dimension=384)
# Call the fit method
try:
  model.fit(
      train_objectives=[(train_dataloader, train_loss)],
      epochs=1,
      # weight_decay=0,
      # scheduler='constantlr', 
      optimizer_params={'lr': 1e-4 ## 3e-5
                        },
      show_progress_bar=True,
      use_amp=True,
      # evaluator=dev_eval, # ORIG - 
      output_path='./cah_sbert_cos',
  )
except:()
"""

"\ntrain_dataset = SentencesDataset(train_samples, model=model)\n# DataLoader to batch your data\ntrain_dataloader = DataLoader(train_dataset, batch_size= 128,#128,\n                              shuffle=True)\n## OnlineContrastiveLoss. - expects pairs\n# train_loss = losses.OnlineContrastiveLoss(model=model, distance_metric=distance_metric, margin=margin)\ntrain_loss = losses.CosineSimilarityLoss(model=model) #ORIG\n\n##ALT:\ntrain_loss = losses.SoftmaxLoss(model=model,num_labels=1, sentence_embedding_dimension=384)\n# Call the fit method\ntry:\n  model.fit(\n      train_objectives=[(train_dataloader, train_loss)],\n      epochs=1,\n      # weight_decay=0,\n      # scheduler='constantlr', \n      optimizer_params={'lr': 1e-4 ## 3e-5\n                        },\n      show_progress_bar=True,\n      use_amp=True,\n      # evaluator=dev_eval, # ORIG - \n      output_path='./cah_sbert_cos',\n  )\nexcept:()\n"

In [26]:
"""
### dev eval
## example: https://jovian.ai/vumichien/sbert
# dev_eval(model) ## 0.555 Accracy (With min 6) , 57% with 1e-3 and min 3
try:dev_eval(model) ## % with 7e-4 and min 4
except:()
"""

'\n### dev eval\n## example: https://jovian.ai/vumichien/sbert\n# dev_eval(model) ## 0.555 Accracy (With min 6) , 57% with 1e-3 and min 3\ntry:dev_eval(model) ## % with 7e-4 and min 4\nexcept:()\n'

In [27]:
%%time
### could speed this up n^2 by getting embeddings only for unique combniations, then rejoining/merging...  (Won't work if using "Text" col)
### ~ 1.5 minutes on colab GPU for 123K train samples , L6.  ~ 30-42 min with L12, full data
### on 2060 local GPU, ~ 19 min for all train
s1_emb = model.encode(list(df[USE_TEXT_COLS[0]].values),
                      normalize_embeddings=False,
                      show_progress_bar=True,
                      batch_size=256,
                    #   convert_to_tensor=True
                    )
if not ONE_COL_DATA_FORMAT:
  ## black cards = quesiton/prompt
  s2_emb = model.encode(list(df[USE_TEXT_COLS[1]].values),
                        normalize_embeddings=False,
                        show_progress_bar=True,
                        batch_size=200,
                      #   convert_to_tensor=True
                      )#.astype(np.float32) ## 1.5 min for 4K , with L12 miniLM, on cpu


Batches:   0%|          | 0/7645 [00:00<?, ?it/s]

CPU times: total: 33min 47s
Wall time: 28min 53s


In [28]:
# try:
#   dot_sim = []
#   cos_score = []
#   for i in range (len(s1_emb)):
#       # dot_sim.append(util.dot_score(s1_emb[i], s2_emb[i])[0])
#       dot_sim.append(float(util.dot_score(s1_emb[i], s2_emb[i])[0])) ## am I getting the right cell with these two?? (cos and dot, [i][j] ? )
#       cos_score.append(float(util.cos_sim(s1_emb[i], s2_emb[i])[0]))
#   print(len(dot_sim))

#   df["dot_sim_score"] = dot_sim
#   df["cos_sim_score"] = cos_score
#   print(df[["dot_sim_score","cos_sim_score","won"]].corr().round(3))

#   print("df:\n",df.groupby("won")[["dot_sim_score","cos_sim_score"]].mean().round(3))

#   # print("df_All:")
#   # df_all["dot_sim_score"] = dot_sim
#   # print(df_all[["dot_sim_score","won"]].corr())
# except:()


## Eval on df_Test
* Can speed up by getting unique sentence pairs and their embeddings
* Won't work the same for single text 

In [29]:
# df_test.drop_duplicates(["black_card_text","white_card_text"]).shape[0] # 367K rows, vs 489 K for all 

In [30]:
%%time
### could speed this up n^2 by getting embeddings only for unique combniations, then rejoining/merging...  (Won't work if using "Text" col)
## change s1_emb to s1_emb_test
## ~ 6 min for L12, local GPU, full test
s1_emb_test = model.encode(list(df_test[USE_TEXT_COLS[0]].values),
                      normalize_embeddings=False,
                      show_progress_bar=True,
                      batch_size=256,
                    #   convert_to_tensor=True
                    )
if not ONE_COL_DATA_FORMAT:
  ## black cards = quesiton/prompt
  s2_emb_test = model.encode(list(df_test[USE_TEXT_COLS[1]].values),
                        normalize_embeddings=False,
                        show_progress_bar=True,
                        batch_size=256,
                      #   convert_to_tensor=True
                      )#.astype(np.float32) ## 1.5 min for 4K , with L12 miniLM, on cpu


Batches:   0%|          | 0/1912 [00:00<?, ?it/s]

CPU times: total: 8min 13s
Wall time: 6min 45s


In [31]:
%%time
if not ONE_COL_DATA_FORMAT:
  try:
    dot_sim = []
    cos_score = []
    for i in range (len(s1_emb)):
        # dot_sim.append(util.dot_score(s1_emb[i], s2_emb[i])[0])
        dot_sim.append(float(util.dot_score(s1_emb[i], s2_emb[i])[0])) ## am I getting the right cell with these two?? (cos and dot, [i][j] ? )
        cos_score.append(float(util.cos_sim(s1_emb[i], s2_emb[i])[0]))
    print(len(dot_sim))

    df_test["dot_sim_score"] = dot_sim
    df_test["cos_sim_score"] = cos_score
    print(df_test[["dot_sim_score","cos_sim_score","won"]].corr().round(3))

    print("df_test:\n",df_test.groupby("won")[["dot_sim_score","cos_sim_score"]].mean().round(3))

    print("Acc:",df_test.sort_values("dot_sim_score",ascending=False).groupby("fake_round_id").head(1)["won"].mean()) ## 17.5% acc
    print("Acc: (dot)",df_test.sort_values("dot_sim_score",ascending=False).groupby("fake_round_id").head(1)["won"].mean())
    print("Acc:",df_test.sort_values("cos_sim_score",ascending=True).groupby("fake_round_id").head(1)["won"].mean()) ## 0.05
    print("Acc:",df_test.sort_values("cos_sim_score",ascending=False).groupby("fake_round_id").head(1)["won"].mean())
  except:()


CPU times: total: 0 ns
Wall time: 0 ns


### Pretrain - unsupervised
* TSDAE or other method ? 
* https://www.sbert.net/examples/unsupervised_learning/TSDAE/README.html#tsdae-as-pre-training-task


ST recommends MultipleNegativesRankingLoss
* https://www.sbert.net/examples/training/nli/README.html#multiplenegativesrankingloss
*  MultipleNegativesRankingLoss only requires positive pairs, i.e., we only need examples of positive/funny pairs. (BUT It also supports hard negatives in a triplet)

Also: OnlineContrastiveLoss
* https://www.sbert.net/examples/training/quora_duplicate_questions/README.html#training
* Constrative Loss / `losses.OnlineContrastiveLoss`
* Choosing the distance function and especially choosing a sensible margin are quite important for the success of constrative loss. In the given example, we use cosine_distance (which is 1-cosine_similarity) with a margin of 0.5. I.e., non-duplicate questions should have a cosine_distance of at least 0.5 (which is equivalent to a 0.5 cosine similarity difference).
* An improved version of constrative loss is OnlineConstrativeLoss, which looks which negative pairs have a lower distance that the largest positive pair and which positive pairs have a higher distance than the lowest distance of negative pairs. I.e., this loss automatically detects the hard cases in a batch and computes the loss only for these cases.

Can also do BOTH losses, as in:

* Multi-Task-Learning
    Constrative Loss works well for pair classification, i.e., given two pairs, are these duplicates or not. It pushes negative pairs far away in vector space, so that the distinguishing between duplicate and non-duplicate pairs works good.

    MultipleNegativesRankingLoss on the other sides mainly reduces the distance between positive pairs out of large set of possible candidates. However, the distance between non-duplicate questions is not so large, so that this loss does not work that weill for pair classification.

* https://www.sbert.net/examples/training/quora_duplicate_questions/README.html#multi-task-learning

More losses (e.g. triplet): https://www.sbert.net/docs/package_reference/losses.html

In [32]:
# # Define your sentence transformer model using CLS pooling
# model_name = 'bert-base-uncased'
# word_embedding_model = models.Transformer(model_name)
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# # Define a list with sentences (1k - 100k sentences)
# train_sentences = ["Your set of sentences",
#                    "Model will automatically add the noise", 
#                    "And re-construct it",
#                    "You should provide at least 1k sentences"]

# # Create the special denoising dataset that adds noise on-the-fly
# train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

# # DataLoader to batch your data
# train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# # Use the denoising auto-encoder loss
# train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

# # Call the fit method
# model.fit(
#     train_objectives=[(train_dataloader, train_loss)],
#     epochs=1,
#     weight_decay=0,
#     scheduler='constantlr',
#     optimizer_params={'lr': 3e-5},
#     show_progress_bar=True
# )

# model.save('output/tsdae-model')

#### get embeddings & dot product/codine distance over all 

* `util.semantic_search` - could do this is one step for us (but we'll want to train a model on embeddings anyway) 

* https://www.sbert.net/docs/usage/semantic_textual_similarity.html

In [33]:
# %%time
# ## white cards = most predictive information

# # ### encode_multi_process ? (needs pool enabled)
# ##     pool = model.start_multi_process_pool(encode_batch_size=2000)
# ##     embeddings = model.encode_multi_process(texts, pool)

# s1_emb = model.encode(list(df["white_card_text"].values),
#                       normalize_embeddings=True,
#                       show_progress_bar=True,
#                       batch_size=128,
#                     #   convert_to_tensor=True 
#                       )#.astype(np.float32) ## 40s for 4K , with L12 miniLM, on cpu
# ### 7 min for all min7 (37k) with L12
# print(s1_emb.shape,"s1_emb.shape")

In [34]:
# %%time
# ### 8.7 min on colab for L12, min 4 (371K)

# s1_emb = model.encode(list(df["white_card_text"].values),
#                       normalize_embeddings=True,
#                       show_progress_bar=True,
#                       batch_size=128,
#                       normalize_embeddings=True
#                     #   convert_to_tensor=True
#                     )

# ## black cards = quesiton/prompt
# s2_emb = model.encode(list(df["black_card_text"].values),
#                       normalize_embeddings=True,
#                       show_progress_bar=True,
#                       batch_size=256,
#                       normalize_embeddings=True
#                     #   convert_to_tensor=True
#                     )#.astype(np.float32) ## 1.5 min for 4K , with L12 miniLM, on cpu
# print(s2_emb.shape)


# dot_sim = []
# for i in range (len(s1_emb)):
#     dot_sim.append(util.dot_score(s1_emb[i], s2_emb[i])[0])
# print(len(dot_sim))
# dot_sim

In [35]:
# len(s2_emb[0]) # 384 = dim of the "first" sample

In [36]:
# ### Hopefully I am doing this correctly ??? 
# ### get cosine similarity 
# output = []
# for i in range (len(s1_emb)):
#     output.append(util.cos_sim(s1_emb[i], s2_emb[i])[0])
# print(len(output))

In [37]:
%%time
#Compute cosine similarity between all pairs - outputs matrix of shape S1 X S2 (i.e # samples = ineffecient in memory!)
# output = util.pytorch_cos_sim(s1_emb, s2_emb)
# output = util.cos_sim(s1_emb, s2_emb) # ORIG, nXn matrix
# print(output.shape)


CPU times: total: 0 ns
Wall time: 0 ns


In [38]:
# %%time
# ### Hopefully I am doing this correctly ??? 
# # # ## dot product:
# # dot_sim = util.dot_score(s1_emb, s2_emb) # memory crash? 
# # print(dot_sim.shape)

# dot_sim = []
# for i in range (len(s1_emb)):
#     dot_sim.append(util.dot_score(s1_emb[i], s2_emb[i])[0])
# print(len(dot_sim))
# dot_sim

In [39]:
# print("cos done")

In [40]:
%%time
# from torch.nn import CosineSimilarity, PairwiseDistance
# ## https://pytorch.org/docs/stable/generated/torch.nn.CosineSimilarity.html
# nn_cos = CosineSimilarity(dim=1, eps=1e-6)
# output = nn_cos(s1_emb, s2_emb) 
# print("output",output.shape)
# nn_pairwiseDist = PairwiseDistance()
# output_2 = nn_pairwiseDist(s1_emb, s2_emb)
# print("output_2",output_2.shape)

# # ## dot product:
# output_3 = util.dot_score(s1_emb, s2_emb)

CPU times: total: 0 ns
Wall time: 0 ns


In [41]:
# df["cos_sim"] = output
# df["pairwiseDist_sim"] = output_2
# df["dot_score"] = output_3

In [42]:
# df.corr()

In [43]:
# df.groupby("picks")[["cos_sim","pairwiseDist_sim"]].mean()

#### Model on embeddings 
* linear model on embeddings per sentence and cossim, pairwise dist score;
* + difference, +- multiplication of vectors

In [44]:
# %%time
# ### mean diff, math/ mult per row: 

# ## https://stackoverflow.com/questions/50430585/mean-difference-of-two-numpy-arrays
# # np.mean(np.abs(s1_emb[:, None] - s2_emb))
# vector_diffs = s1_emb - s2_emb
# mean_diff = np.mean(vector_diffs,axis=1) # 1 col
# max_diff = np.min(vector_diffs,axis=1) 
# min_diff = np.max(vector_diffs,axis=1) 
# # np_dot = np.dot(s1_emb,s2_emb)
# print(mean_diff)
# print(max_diff)

In [45]:
df

,fake_round_id,black_card_text,white_card_text,ID_index,id_white,round_completion_seconds,won,text,sum_won,pair_count
0,141510,"__. Awesome in theory, kind of a mess in pract...",The eight gay warlocks who dictate the rules o...,2,1810,45,0,The eight gay warlocks who dictate the rules o...,0,3
1,29064,I'll take the BBQ bacon burger with a fried eg...,Frantically writing equations on a chalkboard,7,813,22,0,I'll take the BBQ bacon burger with a fried eg...,1,5
2,204306,"I've got rhythm, I've got music, I've got __. ...",The fear and hatred in men's hearts,4,1821,5,0,"I've got rhythm, I've got music, I've got The ...",0,2
3,144997,What never fails to liven up the party?,The best taquito in the galaxy,3,1781,15,0,What never fails to liven up the party? The be...,0,5
4,199,And today's soup is Cream of __.,Taking a shit while running at full speed,5,1700,41,0,And today's soup is Cream of Taking a shit whi...,0,4
...,...,...,...,...,...,...,...,...,...,...
1957075,59190,Just saw this upsetting video! Please retweet!...,Fake tits,9,755,63,0,Just saw this upsetting video! Please retweet!...,1,3
1957076,55070,Oh my god! __ killed Kenny!,Being a total Miranda,7,420,29,0,Oh my god! Being a total Miranda killed Kenny!,0,4
1957077,114606,"Madam President, the asteroid is headed direct...",Telling Heather she can't pull off that top,1,1711,29,0,"Madam President, the asteroid is headed direct...",0,3
1957078,110038,"When I pooped, what came out of my butt?",Eating a pizza that's lying in the street to g...,1,703,19,0,"When I pooped, what came out of my butt? Eatin...",0,2


In [46]:
y = df["won"].values
# X = np.concatenate([s1_emb,s2_emb],axis=1) # BOTH 

X =s1_emb
X_test = s1_emb_test
# try:
#   X = s2_emb # BOTH 
#   X_test = s2_emb_test
# except:
#   X =s1_emb
#   X_test = s1_emb_test
# X = np.concatenate([s1_emb,vector_diffs],axis=1)## ALT - white cards + diffs

print(X.shape)
print(X_test.shape)

(1957080, 768)
(489280, 768)


In [47]:
# X = np.column_stack((X,output)) ### memory crash ? 
# X = np.column_stack((X,dot_sim)) ## dot product 

# # X = np.column_stack((X,output_2))
# # X = np.column_stack((X,output_3))
# X = np.column_stack((X,mean_diff))
# X = np.column_stack((X,max_diff))
# X = np.column_stack((X,min_diff))
# print(X.shape)

In [48]:
%%time
# StandardScaler
clf = make_pipeline(StandardScaler(),  LogisticRegression(solver="sag")) ## PCA(n_components=2), # error with array? 
# clf = make_pipeline(StandardScaler(),  LogisticRegression())
# clf = LogisticRegression() ## same res. ~ 3 min
# cv_preds = cross_val_predict(clf,X,y,n_jobs=-2)

clf.fit(X,y)
y_test_preds = clf.predict(X_test)
print(classification_report(y_true=y_test,y_pred=y_test_preds))
y_test_preds_proba = clf.predict_proba(X_test)[:,1]
print("\nroc_auc",100*round(roc_auc_score(y_true=y_test,y_score=y_test_preds_proba),4))
print("proba based:")
print(classification_report(y_true=y_test,y_pred=y_test_preds_proba>=0.5))

eval_preds(df_test,y_test_preds_proba)
### on sample - roc_auc 59.25 
# """
# cv_preds = cross_val_predict(clf,X,y,method="predict_proba",cv=3,n_jobs=-2)[:,1]
# print(classification_report(y_true=y,y_pred=cv_preds>=0.5))
# print("roc_auc",roc_auc_score(y_true=y,y_score=cv_preds))


# # #### min 4 (joint text), v1, 371K records
# #               precision    recall  f1-score   support

# #            0       0.65      0.90      0.75    227115
# #            1       0.58      0.22      0.32    144299

# #     accuracy                           0.64    371414
# #    macro avg       0.61      0.56      0.54    371414
# # weighted avg       0.62      0.64      0.58    371414

# # roc_auc 0.620


# """

C:\apps\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\apps\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\apps\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.90      1.00      0.95    440352
           1       0.00      0.00      0.00     48928

    accuracy                           0.90    489280
   macro avg       0.45      0.50      0.47    489280
weighted avg       0.81      0.90      0.85    489280


roc_auc 59.040000000000006
proba based:


C:\apps\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\apps\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\apps\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.90      1.00      0.95    440352
           1       0.00      0.00      0.00     48928

    accuracy                           0.90    489280
   macro avg       0.45      0.50      0.47    489280
weighted avg       0.81      0.90      0.85    489280

              precision    recall  f1-score   support

           0       0.92      1.00      0.96    440352
           1       1.00      0.17      0.29     48928

    accuracy                           0.92    489280
   macro avg       0.96      0.58      0.62    489280
weighted avg       0.92      0.92      0.89    489280

rocAUC: 0.5904
top 1 acc by round: 16.90
top 2 acc by round: 15.34
top 3 acc by round: 14.22
top 1 acc by round: 0.169
top 2 acc by round: 0.3068
top 3 acc by round: 0.4266
CPU times: total: 10min 11s
Wall time: 11min 40s


* RES:


              precision    recall  f1-score   support

           0       0.90      1.00      0.95    440352
           1       0.00      0.00      0.00     48928

    accuracy                           0.90    489280
   macro avg       0.45      0.50      0.47    489280
weighted avg       0.81      0.90      0.85    489280


roc_auc 59.040000000000006

proba based:

              precision    recall  f1-score   support

           0       0.90      1.00      0.95    440352
           1       0.00      0.00      0.00     48928

    accuracy                           0.90    489280
   macro avg       0.45      0.50      0.47    489280
weighted avg       0.81      0.90      0.85    489280

              precision    recall  f1-score   support

           0       0.92      1.00      0.96    440352
           1       1.00      0.17      0.29     48928

    accuracy                           0.92    489280
   macro avg       0.96      0.58      0.62    489280
weighted avg       0.92      0.92      0.89    489280

rocAUC: 0.5904
top 1 acc by round: 16.90
top 2 acc by round: 15.34
top 3 acc by round: 14.22
top 1 acc by round: 0.169
top 2 acc by round: 0.3068
top 3 acc by round: 0.4266

In [49]:
%%time
## slow... ?  30 min
clf_rf = RandomForestClassifier(min_samples_split=5, min_samples_leaf=12,#,max_depth=9,
                                n_jobs=-2,ccp_alpha=0.005) # n_jobs=-2

# cv_preds_2 = cross_val_predict(clf_rf,X,y,method="predict_proba",cv=3)[:,1]
# print(classification_report(y_true=y,y_pred=cv_preds_2>=0.5))
# print("roc_auc",roc_auc_score(y_true=y,y_score=cv_preds_2))

clf_rf.fit(X,y)
print("Trained")
y_test_preds_rf = clf_rf.predict(X_test)
print(classification_report(y_true=y_test,y_pred=y_test_preds_rf))
y_test_preds_proba_rf = clf_rf.predict_proba(X_test)[:,1]
print("\nroc_auc",100*round(roc_auc_score(y_true=y_test,y_score=y_test_preds_proba_rf),4))
print("proba based:")
print(classification_report(y_true=y_test,y_pred=y_test_preds_proba_rf>=0.5))

eval_preds(df_test,y_test_preds_proba_rf)
# # #### min 4 (joint text), v1, 371K records
#               precision    recall  f1-score   support

#            0       0.63      0.94      0.76    227115
#            1       0.61      0.14      0.23    144299

#     accuracy                           0.63    371414
#    macro avg       0.62      0.54      0.49    371414
# weighted avg       0.62      0.63      0.55    371414

# roc_auc 0.609



##TODO - feature importance (do the deltas improve model?)

# min 2 , L12 miniLM
#               precision    recall  f1-score   support

#            0       0.73      0.97      0.83    469455
#            1       0.53      0.10      0.17    183561

#     accuracy                           0.72    653016
#    macro avg       0.63      0.53      0.50    653016
# weighted avg       0.68      0.72      0.65    653016

# roc_auc 0.64871
# CPU times: user 2h 

Trained


C:\apps\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\apps\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\apps\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.90      1.00      0.95    440352
           1       0.00      0.00      0.00     48928

    accuracy                           0.90    489280
   macro avg       0.45      0.50      0.47    489280
weighted avg       0.81      0.90      0.85    489280


roc_auc 50.0
proba based:


C:\apps\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\apps\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\apps\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.90      1.00      0.95    440352
           1       0.00      0.00      0.00     48928

    accuracy                           0.90    489280
   macro avg       0.45      0.50      0.47    489280
weighted avg       0.81      0.90      0.85    489280

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    440352
           1       1.00      1.00      1.00     48928

    accuracy                           1.00    489280
   macro avg       1.00      1.00      1.00    489280
weighted avg       1.00      1.00      1.00    489280

rocAUC: 0.5
top 1 acc by round: 10.03
top 2 acc by round: 9.98
top 3 acc by round: 9.94
top 1 acc by round: 0.1003
top 2 acc by round: 0.1995
top 3 acc by round: 0.2983
CPU times: total: 9h 14min 58s
Wall time: 1h 25min 44s


In [50]:
eval_preds(df_test,y_test_preds_proba_rf)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    440352
           1       1.00      1.00      1.00     48928

    accuracy                           1.00    489280
   macro avg       1.00      1.00      1.00    489280
weighted avg       1.00      1.00      1.00    489280

rocAUC: 0.5
top 1 acc by round: 10.09
top 2 acc by round: 10.00
top 3 acc by round: 9.97
top 1 acc by round: 0.1009
top 2 acc by round: 0.2001
top 3 acc by round: 0.2992


RES: (Random forest):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    440352
           1       1.00      1.00      1.00     48928

    accuracy                           1.00    489280
   macro avg       1.00      1.00      1.00    489280
weighted avg       1.00      1.00      1.00    489280

rocAUC: 0.5
top 1 acc by round: 10.09
top 2 acc by round: 10.00
top 3 acc by round: 9.97
top 1 acc by round: 0.1009
top 2 acc by round: 0.2001
top 3 acc by round: 0.2992

In [51]:
df["text"].str.split().str.len().max()

55

In [52]:
from sklearn.model_selection import train_test_split
# y = df["picks"].values
# # X = np.concatenate([s1_emb,s2_emb],axis=1) # BOTH 
# X = list(df["black_card_text"].values)
"""
##ORIG:
X_train, X_test, y_train, y_test = train_test_split(
    list(df["text"].values), list(df["won"].values), test_size=0.25, random_state=42)
"""
## NEW:


'\n##ORIG:\nX_train, X_test, y_train, y_test = train_test_split(\n    list(df["text"].values), list(df["won"].values), test_size=0.25, random_state=42)\n'

In [53]:
set(y_train)

{0, 1}

In [54]:
# Try finetuning? 

In [57]:
### https://www.sbert.net/docs/training/overview.html
from torch import nn

word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(),
                           out_features=64, # 128
                           activation_function=nn.Tanh())

# model2 = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
model2 = SentenceTransformer(modules=[model, dense_model])

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [58]:
len(y_train)

1957080

In [59]:
# [X_train[0:5],y_train[0:5]]
## list(df["black_card_text"].values)
train_examples =  [InputExample(texts=[X_train[i]],label=float(y_train[i])) for i in range(len(X_train)-1)]
# [X_train[0:5],y_train[0:5]]

In [61]:
X_train[0]

'The eight gay warlocks who dictate the rules of fashion. Awesome in theory, kind of a mess in practice.'

In [62]:
len(y_train)

1957080

In [63]:
len(X_train)

1957080

In [64]:
len(train_examples)

1957079

In [65]:
### IndexError: list index out of range. ? 


#Define your train examples. You need more than just two examples...
# train_examples = [InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
#     InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]

# train_examples =  [X_train[0:5],y_train[0:5]]

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=128)
# train_loss = losses.CosineSimilarityLoss(model)

# #Tune the model
# model2.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=10)

In [66]:
%%time
### https://www.sbert.net/docs/package_reference/losses.html#batchsemihardtripletloss
## supports 1 sentence/label pair
train_dataset = SentencesDataset(train_examples, model2)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=256)
train_loss = losses.BatchSemiHardTripletLoss(model=model2)

#Tune the model
model2.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2, warmup_steps=50)

C:\apps\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/7645 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 36.00 MiB (GPU 0; 6.00 GiB total capacity; 5.14 GiB already allocated; 0 bytes free; 5.30 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF